In [1]:
import __future__


In [2]:
from __future__ import print_function
from __future__ import division

In [3]:
#utils
import numpy as np
import scipy.sparse as sp
import torch

def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),dtype=np.int32)
    return labels_onehot

   #onehot vector for 7 classes in output

def load_data(path="", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),dtype=np.dtype(str))
    
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    #csr=Compressed Sparse Row 
    #array  of rows
    
    labels = encode_onehot(idx_features_labels[:, -1])
    #last col  of idx_features_labels=7 classes
    
    # build graph
    
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    #first col in every row=doc id
    
    idx_map = {j: i for i, j in enumerate(idx)}
    #{'31336': 0, '1061127':1, ...}
    
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)
    #each doc is connected to which doc (citation) 

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    #idx_train = range(2)
    #idx_val = range(2, 3)
    #idx_test = range(4)
    
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 501)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    #sum in every row 
    r_inv = np.power(rowsum, -1).flatten()
    # every sum to the power of -1 
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    #diagonal matrice 
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


In [4]:
# Load data
#idx_features_labels= load_data()
#idx_features_labels


In [5]:
#labels= load_data()
#labels

In [6]:
#labels.shape

In [7]:
#features= load_data()
#features

In [8]:
#idx=load_data()
#idx

In [9]:
#idx_map=load_data()
#idx_map

In [10]:
#edges_unordered=load_data()
#edges_unordered

In [11]:
#edges=load_data()
#edges

In [12]:
#adj=load_data()
#adj

In [13]:
#layers

import math

import torch

from torch.nn.parameter import Parameter

#class parameter ,param haro cache mikone

from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        
        #Sparse matrix multiplication=spmm
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ') '

In [14]:
#models
import torch.nn as nn
import torch.nn.functional as F
from layers import GraphConvolution


class GCN(nn.Module):
    def __init__(self, nfeat, nhid,nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        #self.gc2 = GraphConvolution(nhid, nhid2)
        #self.gc3 = GraphConvolution(nhid2, nclass)
        
        #adding a 3th layer didn't outperform
        
        self.dropout = dropout
        
    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        #can also use tanh,not very different result
        
        x = F.dropout(x, self.dropout, training=self.training)
        #return F.log_softmax(x, dim=1)
        #print(x.size())
    
        x = self.gc2(x, adj)
        
        #x = self.gc3(x, adj)
        #return x
       # print(x.size())
        return F.log_softmax(x, dim=1)


In [15]:
#train

from __future__ import division
from __future__ import print_function
from torchsummary import summary

import time
import argparse
import numpy as np

#import torch
import torch.nn.functional as F
import torch.optim as optim


#from utils import load_data, accuracy
#from models import GCN

# Training settings

parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')

#parser.add_argument('--hidden2', type=int, default=8,
                    #help='Number of hidden units.')

parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

#args = parser.parse_args()
#error midad khate bala,khate paein jaigozin shod

args = parser.parse_known_args()[0]

args.cuda = not args.no_cuda and torch.cuda.is_available()

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            #nhid2=args.hidden2,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

## to set cuda as your device if possible
##training on  GPU

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()
    
    ## train:adjust the weights on the neural network
    ## validation:used to minimize overfitting

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val])
        acc_val = accuracy(output[idx_val], labels[idx_val])
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    ## Turn on evaluation mode which disables dropout.
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

print(model)

summary(model,[(2708,1433),(2708,2708)])
# Testing
test()




Loading cora dataset...
Epoch: 0001 loss_train: 1.9318 acc_train: 0.2143 loss_val: 1.9087 acc_val: 0.3500 time: 0.0899s
Epoch: 0002 loss_train: 1.9290 acc_train: 0.2929 loss_val: 1.8981 acc_val: 0.3500 time: 0.0819s
Epoch: 0003 loss_train: 1.9186 acc_train: 0.2929 loss_val: 1.8884 acc_val: 0.3500 time: 0.0770s
Epoch: 0004 loss_train: 1.9057 acc_train: 0.2929 loss_val: 1.8791 acc_val: 0.3500 time: 0.0859s
Epoch: 0005 loss_train: 1.8902 acc_train: 0.2929 loss_val: 1.8702 acc_val: 0.3500 time: 0.0849s
Epoch: 0006 loss_train: 1.8876 acc_train: 0.2929 loss_val: 1.8615 acc_val: 0.3500 time: 0.0820s
Epoch: 0007 loss_train: 1.8865 acc_train: 0.2929 loss_val: 1.8531 acc_val: 0.3500 time: 0.1019s
Epoch: 0008 loss_train: 1.8740 acc_train: 0.2929 loss_val: 1.8449 acc_val: 0.3500 time: 0.0790s
Epoch: 0009 loss_train: 1.8580 acc_train: 0.2929 loss_val: 1.8368 acc_val: 0.3500 time: 0.0829s
Epoch: 0010 loss_train: 1.8539 acc_train: 0.2929 loss_val: 1.8288 acc_val: 0.3500 time: 0.0820s
Epoch: 0011 loss

In [16]:
adj

tensor(indices=tensor([[   0,    8,   14,  ..., 1389, 2344, 2707],
                       [   0,    0,    0,  ..., 2707, 2707, 2707]]),
       values=tensor([0.1667, 0.1667, 0.0500,  ..., 0.2000, 0.5000, 0.2500]),
       size=(2708, 2708), nnz=13264, layout=torch.sparse_coo)

In [17]:
features.shape

torch.Size([2708, 1433])

In [18]:
set(labels)

{tensor(3),
 tensor(1),
 tensor(5),
 tensor(5),
 tensor(1),
 tensor(4),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(5),
 tensor(1),
 tensor(4),
 tensor(4),
 tensor(4),
 tensor(4),
 tensor(1),
 tensor(5),
 tensor(5),
 tensor(1),
 tensor(6),
 tensor(5),
 tensor(3),
 tensor(1),
 tensor(4),
 tensor(1),
 tensor(0),
 tensor(3),
 tensor(4),
 tensor(3),
 tensor(5),
 tensor(1),
 tensor(5),
 tensor(0),
 tensor(1),
 tensor(4),
 tensor(1),
 tensor(5),
 tensor(0),
 tensor(1),
 tensor(3),
 tensor(1),
 tensor(1),
 tensor(4),
 tensor(5),
 tensor(0),
 tensor(1),
 tensor(2),
 tensor(1),
 tensor(2),
 tensor(6),
 tensor(3),
 tensor(2),
 tensor(4),
 tensor(5),
 tensor(0),
 tensor(3),
 tensor(1),
 tensor(2),
 tensor(5),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(6),
 tensor(6),
 tensor(0),
 tensor(3),
 tensor(3),
 tensor(5),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(5),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(4),
 tensor(2),
 tensor(1),
 tensor(4),
 tensor(2),
 tensor(0),
 tensor(5),
 ten

In [19]:
adj.shape

torch.Size([2708, 2708])

In [20]:
features

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [21]:
labels

tensor([1, 2, 6,  ..., 4, 0, 1])

In [22]:
idx_train




tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139])

In [23]:
 idx_val

tensor([200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
        214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227,
        228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241,
        242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255,
        256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269,
        270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283,
        284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297,
        298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311,
        312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325,
        326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339,
        340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353,
        354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367,
        368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 3

In [24]:
 idx_test

tensor([500])